### Imports (make sure to install janus)

In [ ]:
!pip install -r requirements.txt

In [ ]:
# !pip install -e ..

In [ ]:
!pip show janus

In [ ]:
!pip install pyspark

In [3]:
import janus
from janus.stats.experiment import Experiment, Variant
import pandas as pd

import logging
logging.basicConfig(level=logging.INFO)

## Load Test Dataset

In [6]:
df = pd.read_csv("examples/results_per_user.csv")
df

,id,alternative,revenue,sum_revenue,sales
0,1,A,0,0,0
1,2,A,0,0,0
2,3,A,0,0,0
3,4,A,100,100,1
4,5,A,200,200,1
5,6,A,300,300,1
6,7,B,0,0,0
7,8,B,230,230,1
8,9,B,170,170,1
9,10,B,320,320,1


We see that, for the data we have, alternative B looks better: 3 sales out of 4 participants, with average ticket of (230+170+320)/3 = 240 and average ARPU of 180. All metrics are higher that A's.

## Initialize experiment

In [7]:
experiment = Experiment(name="my_experiment",
                       keymetrics=['conversion', 'revenue', 'arpu'],
                       baseline_variant_name='A')

We run the experiment over our data to get results

In [8]:
experiment.run_experiment(df_results_per_user=df)

INFO:root:INITIALIZING experiment 'my_experiment' evaluation...
INFO:root:consolidating metrics over variants...
INFO:root:calculating bayesian statistics over variants for metric conversion
INFO:root:sampling data for variant A...
INFO:root:sampling data for variant B...
INFO:root:calculating probability to beat and expected loss for variant A...
INFO:root:calculating probability to beat and expected loss for variant B...
INFO:root:calculating bayesian statistics over variants for metric revenue
INFO:root:sampling data for variant A...
INFO:root:sampling data for variant B...
INFO:root:calculating probability to beat and expected loss for variant A...
INFO:root:calculating probability to beat and expected loss for variant B...
INFO:root:calculating bayesian statistics over variants for metric arpu
INFO:root:sampling data for variant A...
INFO:root:sampling data for variant B...
INFO:root:calculating probability to beat and expected loss for variant A...
INFO:root:calculating probabili

{'A': {'users': 6,
  'sales': 3,
  'paids': 3,
  'revenue': 600,
  'conversion': 0.5,
  'ticket': 200.0,
  'arpu': 100.0,
  'statistics': {'conversion': {'chance_to_beat': 0.246727,
    'expected_loss': 0.20485381083632845,
    'lift': -0.33333333333333337,
    'diff': -0.25},
   'revenue': {'chance_to_beat': 0.401356,
    'expected_loss': 90.26219646796406,
    'lift': -0.16666666666666663,
    'diff': -120},
   'arpu': {'chance_to_beat': 0.289212,
    'expected_loss': 81.58532656393245,
    'lift': -0.4444444444444444,
    'diff': -80.0}},
  'ratio': 0.6},
 'B': {'users': 4,
  'sales': 3,
  'paids': 3,
  'revenue': 720,
  'conversion': 0.75,
  'ticket': 240.0,
  'arpu': 180.0,
  'statistics': {'conversion': {'chance_to_beat': 0.753273,
    'expected_loss': 0.03789908703261173,
    'lift': 0.5,
    'diff': 0.25},
   'revenue': {'chance_to_beat': 0.598644,
    'expected_loss': 50.334123257073756,
    'lift': 0.19999999999999996,
    'diff': 120},
   'arpu': {'chance_to_beat': 0.710788,

In [9]:
pd.DataFrame.from_dict(experiment.results['A']['statistics'])

,conversion,revenue,arpu
chance_to_beat,0.246727,0.401356,0.289212
expected_loss,0.204854,90.262196,81.585327
lift,-0.333333,-0.166667,-0.444444
diff,-0.250000,-120.000000,-80.000000


In [10]:
pd.DataFrame.from_dict(experiment.results)

,A,B
users,6,4
sales,3,3
paids,3,3
revenue,600,720
conversion,0.5,0.75
ticket,200.0,240.0
arpu,100.0,180.0
statistics,"{'conversion': {'chance_to_beat': 0.246727, 'e...","{'conversion': {'chance_to_beat': 0.753273, 'e..."
ratio,0.6,0.4


In [20]:
_df = pd.DataFrame.from_dict(experiment.results).drop('statistics')

_df.T

,users,sales,paids,revenue,conversion,ticket,arpu,ratio
A,6,3,3,600,0.5,200.0,100.0,0.6
B,4,3,3,720,0.75,240.0,180.0,0.4


In [17]:
statsA = pd.DataFrame.from_dict(experiment.results['A']['statistics'])
statsB = pd.DataFrame.from_dict(experiment.results['B']['statistics'])

statsA

,conversion,revenue,arpu
chance_to_beat,0.246727,0.401356,0.289212
expected_loss,0.204854,90.262196,81.585327
lift,-0.333333,-0.166667,-0.444444
diff,-0.250000,-120.000000,-80.000000


## References to compare

https://vidogreg.shinyapps.io/bayes-arpu-test/

https://abtestguide.com/bayesian/